In [ ]:
%AddJar https://repo1.maven.org/maven2/org/elasticsearch/elasticsearch-spark-20_2.11/7.7.0/elasticsearch-spark-20_2.11-7.7.0.jar

In [ ]:
val spark = SparkSession
  .builder()
  .appName("Lab33")
  .getOrCreate()
import spark.implicits._

In [ ]:
val esOptions =
  Map(
    "es.nodes" -> "10.0.0.5:9200",
    "es.batch.write.refresh" -> "false",
    "es.nodes.wan.only" -> "true",
    "es.net.http.auth.user" -> "andrey.blednykh2",
    "es.net.http.auth.pass" -> "AkRjOhAK"
  )

In [ ]:
var logs = spark.read.format("org.elasticsearch.spark.sql").options(esOptions).load("visits*")

In [ ]:
logs.show(2, 200, true)

In [ ]:
import org.apache.spark.sql.functions._

In [ ]:
val visitsByCategory = logs
    .withColumn("all_shop_cat",    
        concat(lit("shop_"), regexp_replace(lower(col("category")), "[-,\\s]", "_")))
    .select("uid", "all_shop_cat")
    .na.drop(Seq("uid"))
    .groupBy("uid")
    .pivot("all_shop_cat")
    .agg(count("all_shop_cat"))
    .na.fill(0)

In [ ]:
visitsByCategory.show(2, 200, true)